In [6]:
#train3 from 2: use adam, remove weight_decay; one BW channel (not 3)
import torch,os,glob
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision.transforms import v2
from torchvision import tv_tensors
import pandas as pd
import numpy as np

from nets import ResNet18_3lbC_nlbM
from tqdm import tqdm
from configparser import ConfigParser
from torch.utils.data import  DataLoader
from LIDC_M_data import LIDC_Dataset
import pickle

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
best_epoch = 0
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

prep_tr = [
    v2.Lambda(lambda x: tv_tensors.Image(torch.clamp(x,-1000.,400.)) if isinstance(x, tv_tensors.Image) else x),
    v2.Lambda(lambda x: tv_tensors.Image((x+1000)/1400) if isinstance(x, tv_tensors.Image) else x),
    v2.CenterCrop((384,384)),
]
aug_tr = [
    v2.RandomAffine(degrees=10),
    v2.RandomHorizontalFlip(),
]
trans_train = v2.Compose( prep_tr + aug_tr )
trans_test = v2.Compose( prep_tr  )

In [8]:

parser = ConfigParser()
parser.read('.settings')
root_dir = parser.get('dataset','root_dir') #/workspaces/data/lidc-idri/slices
meta_dir = parser.get('dataset','meta_dir') #/workspaces/data/lidc-idri/splits
result_dir = os.path.join(parser.get('dataset','result_dir'),'stage2/basel0_3lbC_2lbM')
if not os.path.isdir(result_dir):
        os.mkdir(result_dir)

train_data = LIDC_Dataset(root_dir,metapath=os.path.join(meta_dir,'trainBB_malB.csv'),transform=trans_train, loadBB=True)
test_data = LIDC_Dataset(root_dir,metapath=os.path.join(meta_dir,'testBB_malB.csv'),transform=trans_test)
total_train_data = len(train_data)
total_test_data = len(test_data)
print('total_train_data:',total_train_data, 'total_test_data:',total_test_data)

batch_size = int(parser['dataset']['batch_size'])
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
testloader = DataLoader(test_data, batch_size=batch_size, num_workers=8)

total_train_data: 5495 total_test_data: 2354


In [9]:
net = ResNet18_3lbC_nlbM(pretrained=True,attr="MGA2")
net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
net.fc = nn.Linear(net.fc.in_features, 2)
net = net.to(device)

In [10]:
from torchinfo import summary
summary(net, input_size=(batch_size,1, 384, 384))

Layer (type:depth-idx)                             Output Shape              Param #
ResNet18_3lbC_nlbM                                 [32, 2]                   --
├─Conv2d: 1-1                                      [32, 64, 192, 192]        3,136
├─BatchNorm2d: 1-2                                 [32, 64, 192, 192]        128
├─ReLU: 1-3                                        [32, 64, 192, 192]        --
├─MaxPool2d: 1-4                                   [32, 64, 96, 96]          --
├─Sequential: 1-5                                  [32, 64, 96, 96]          --
│    └─BasicBlock: 2-1                             [32, 64, 96, 96]          --
│    │    └─Conv2d: 3-1                            [32, 64, 96, 96]          36,864
│    │    └─BatchNorm2d: 3-2                       [32, 64, 96, 96]          128
│    │    └─ReLU: 3-3                              [32, 64, 96, 96]          --
│    │    └─Conv2d: 3-4                            [32, 64, 96, 96]          36,864
│    │    └─BatchNorm2

In [11]:
lr = 1e-4
criterion = nn.CrossEntropyLoss()
mse = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

training_info=[["epoch","acc","loss"]]
testing_info=training_info.copy()

In [12]:
def train(epoch):
    net.train()
    train_loss = np.zeros(3)
    correct = 0
    total = 0
    pbar = tqdm(trainloader)
    for batch_idx, (inputs, targets, masks) in enumerate(pbar):
        inputs, targets, masks = inputs.to(device), targets.to(device), masks.to(device)

        optimizer.zero_grad()
        # outputs = net(inputs)
        outputs, attn_map = net(inputs)
        cls_loss = criterion(outputs, targets)
        masks = F.adaptive_avg_pool2d(masks, attn_map.shape[-2:])
        att_loss = 100.* mse(attn_map , masks)
        
        loss =  cls_loss + att_loss
        
        # loss = criterion(outputs, targets)
        loss.backward()
        
        optimizer.step()

        train_loss += np.array([loss.item(), cls_loss.item(), att_loss.item()])
        _, predicted = outputs.max(1)
        
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        pbar.set_description(f"Epoch: {epoch} Acc: {(100.*correct/total):.2f}")

    train_acc = 100.*correct/total
    train_loss = train_loss/(batch_idx+1)
    print(f"Tot Loss: {train_loss[0]:.4f} CL: {train_loss[1]:.5f} AT: {train_loss[2]:.5f}; Train Acc: {train_acc:.2f}%")
    training_info.append([epoch,train_acc,train_loss])
    # trainning_accuracy.append(train_acc)
    # trainning_loss.append( train_loss )

def test(epoch, islast = False):
    global best_acc, best_epoch
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    pbar = tqdm(testloader)
    with torch.no_grad():
        for batch_idx, (inputs, targets ) in enumerate(pbar):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        test_acc = 100.*correct/total
        test_loss = test_loss/(batch_idx+1)
        print(f"Test Loss: {test_loss}, Test Acc: {test_acc:.2f}%")
        testing_info.append([epoch,test_acc,test_loss])
        # testing_accuracy.append(test_acc)
        # testing_loss.append(test_loss)
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc or islast:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        savestr = 'best' if acc > best_acc else 'last'
        torch.save(state, os.path.join(result_dir,f'basel0-b{batch_size}-epoch{epoch}-{savestr}.pth'))
        best_acc = acc
        best_epoch = epoch

In [13]:
aa = np.zeros(3)
aa = [1,2,3]

In [14]:
print(aa)

[1, 2, 3]


In [15]:
if start_epoch>0:
    checkpoint = torch.load(glob.glob(os.path.join(result_dir,f'basel0-b{batch_size}-epoch{start_epoch-1}-*.pth'))[0] )
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']

for epoch in range(start_epoch, start_epoch+50):
    train(epoch)
    test(epoch, islast = epoch==start_epoch+49)

Epoch: 0 Acc: 57.71: 100%|██████████| 172/172 [00:43<00:00,  3.94it/s]


Tot Loss: 22.6644 CL: 0.68351 AT: 21.98084; Train Acc: 57.71%


100%|██████████| 74/74 [00:06<00:00, 12.10it/s]


Test Loss: 0.670093152974103, Test Acc: 58.54%
Saving..


Epoch: 1 Acc: 61.73: 100%|██████████| 172/172 [00:42<00:00,  4.02it/s]


...

Test Loss: 0.5295427970185473, Test Acc: 86.15%


Epoch: 49 Acc: 96.94: 100%|██████████| 172/172 [00:46<00:00,  3.72it/s]


Tot Loss: 0.1384 CL: 0.08076 AT: 0.05761; Train Acc: 96.94%


100%|██████████| 74/74 [00:09<00:00,  7.57it/s]


Test Loss: 0.5191629845649004, Test Acc: 85.98%
Saving..


In [16]:
traindf = pd.DataFrame(training_info[1:],columns=training_info[0])
testdf = pd.DataFrame(testing_info[1:],columns=testing_info[0])
with open(os.path.join(result_dir,f'basel0-b{batch_size}-info.pkl'), 'wb') as file:
    pickle.dump({"train":traindf,"test":testdf}, file)

 
 #   scheduler.step()

In [17]:
import re
levels = re.findall(r'MGA(\d+)','MGA22')
mga_level = int(levels[0]) if levels else 3
# 'MGA3'.startswith('MGA')
mga_level

22

In [18]:
testdf

,epoch,acc,loss
0,0,58.538658,0.670093
1,1,64.401020,0.630775
2,2,63.381478,0.637113
3,3,66.822430,0.605163
4,4,69.243840,0.587622
5,5,71.197961,0.562709
6,6,72.854715,0.542424
7,7,71.325404,0.607925
8,8,73.449448,0.546692
9,9,74.596432,0.527992


In [19]:
# with open('two_dfs.pkl', 'rb') as file:
#     loaded = pickle.load(file)

# loaded